# Install Dependencies

In [ ]:
%pip install tensorflow
%pip install tensorflow-gpu
%pip install tensorflow-hub
%pip install opencv-python
%pip install matplotlib
%pip install sklearn

# Import Dependencies

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

In [ ]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load Movenet Model

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# Make Detections

In [ ]:
# Fucntion that draws points of the joints on the screen
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [ ]:
# object that mapes the connections between each point drawn on the screen
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
# Function that draws the edges connecting the points drwan by draw_keypoints fucntion
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [ ]:
# Function to loop through each person detected and render with maximum number 6 persons
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Open Camera and Render Skeleton

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0),192,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.35)
    
    cv2.imshow('Human Gait Identification', frame)
    
    if cv2.waitKey(10) & 0xFF== 27:
        break
cap.release()
cv2.destroyAllWindows()

# Folder Setup for Data Collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Custom_Gait_Dataset') 

# Persons that we try to detect
persons = np.array(['Lance','Seif'])

#Very person with 4 different view
views = np.array(['Front','Back','Left','Right'])

# 10 videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 60

In [ ]:
# for person in persons:
#     for view in views:
#         for sequence in range(no_sequences):
#             try: 
#                 os.makedirs(os.path.join(DATA_PATH, person, view, str(sequence)))
#             except:
#                 pass

# Dataset Collection

In [ ]:
# cap = cv2.VideoCapture(1)
# for person in persons:
#     for view in views:
#         for sequence in range(no_sequences):
#             for frame_num in range(sequence_length):

#                 ret, frame = cap.read()
                    
#                 # Resize image
#                 image = frame.copy()
#                 image = tf.image.resize_with_pad(tf.expand_dims(image, axis=0),192,256)
#                 input_img = tf.cast(image, dtype=tf.int32)
                    
#                 # Detection section
#                 results = movenet(input_img)
#                 extracted_keypoints = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
                    
#                 # Render keypoints 
#                 loop_through_people(frame, extracted_keypoints, EDGES, 0.35)

#                 if frame_num == 0: 
#                     cv2.putText(frame, 'Starting {} Collection'.format(person), (120,200), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
#                     cv2.putText(frame, 'Collecting frames for {} with {} view, Video Number {}'.format(person, view, sequence), (15,12), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.waitKey(3000)
#                 else: 
#                     cv2.putText(frame, 'Capturing'.format(person), (120,200), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
#                     cv2.putText(frame, 'Collecting frames for {} with {} view, Video Number {}'.format(person, view, sequence), (15,12), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        
#                 # NEW Export keypoints
#                 keypoints = extracted_keypoints
#                 npy_path = os.path.join(DATA_PATH, person, view, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
                    
#                 cv2.imshow('Human Gait Identification', frame)
        
#                 if cv2.waitKey(10) & 0xFF== 27:
#                     break
# cap.release()
# cv2.destroyAllWindows()

# Creating labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(persons)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for person in persons:
    for view in views:
        for sequence in np.array(os.listdir(os.path.join(DATA_PATH, person, view))).astype(int):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH, person, view, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[person])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

# CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 17, 3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(persons.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

In [ ]:
model.summary()